In [1]:
%load_ext lab_black

In [2]:
from pathlib import Path

import contextily as ctx
import geopandas as gpd
from joblib import Parallel, delayed

In [3]:
root = Path("..", "data")
out_path = Path(root, "outputs")
img_path = Path(root, "figures")

In [4]:
east_coast = gpd.read_file(Path(out_path, "east_coast.gpkg")).to_crs("EPSG:3857")
coords = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(*zip(*east_coast.geometry[0].coords)),
    crs=east_coast.crs,
)
coords["distance"] = coords.geometry.distance(coords.geometry.shift(1)).fillna(0.0)
coords["buff"] = 0.5 * (coords["distance"] + coords["distance"].shift(-1).fillna(0.0))
coords["bbox"] = coords.geometry.buffer(coords["buff"])
bboxes = coords.bbox.bounds

In [5]:
def get_stamen(west, south, east, north, path, style="TonerBackground"):
    source = getattr(ctx.providers.Stamen, style)
    if not path.exists():
        ctx.bounds2raster(
            west,
            south,
            east,
            north,
            path,
            source=source,
        )

In [6]:
fpath = Path(img_path, "coast_images")
_ = Parallel(n_jobs=12)(
    delayed(get_stamen)(w, s, e, n, Path(fpath, f"image_{i}.tif"))
    for i, (w, s, e, n) in bboxes.iterrows()
)